## Classification of Income Being Above or Below/Equal to $50,000 in Adult Census Data

By: Sunsar, Sarah, Emily, Calvin (DSCI 100 003 - Group 23)

Data is from: https://www.kaggle.com/datasets/uciml/adult-census-income

The dataset used for this analysis is derived from the 1994 Census Bureau database. 

The dataset contains a diverse range of numerical and categorical attributes, such as age, hours worked per week,  sex, and more. In this project, we will filter and simplify some categories from this dataset to predict whether an individual"s annual salary falls above or below $50,000.

The question this project will seek to answer is: 

**How do different aspects of a person predict whether annual income will be above or below $50K annually?**

# Preliminary Exploratory Data Analysis

In [2]:
library(tidyverse)
library(tidymodels)

-- Attaching core tidyverse packages ------------------------ tidyverse 2.0.0 --
v dplyr     1.1.3     v readr     2.1.4
v forcats   1.0.0     v stringr   1.5.0
v ggplot2   3.4.3     v tibble    3.2.1
v lubridate 1.9.2     v tidyr     1.3.0
v purrr     1.0.2     
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()
i Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
-- Attaching packages -------------------------------------- tidymodels 1.1.1 --

v broom        1.0.5     v rsample      1.2.0
v dials        1.2.0     v tune         1.1.2
v infer        1.0.5     v workflows    1.1.3
v modeldata    1.2.0     v workflowsets 1.0.1
v parsnip      1.1.1     v yardstick    1.2.0
v recipes      1.0.8     

-- Conflicts ----------------------------------------- tidymodels_conflicts() --
x scales::discard() masks purrr::discard()
x dplyr::filt

In [3]:
df = read_csv("https://raw.githubusercontent.com/calvingdu/dsci100-003-23/master/data/adult_census.csv")

# Splitting the data
df_split <- initial_split(df, prop = 0.8, strata = income)
df_train <- training(df_split)
df_test <- testing(df_split)

paste0("Training set row count: ", nrow(df_train))
paste0("Testing set row count: ", nrow(df_test))

Rows: 32561 Columns: 15
-- Column specification --------------------------------------------------------
Delimiter: ","
chr (9): workclass, education, marital.status, occupation, relationship, rac...
dbl (6): age, fnlwgt, education.num, capital.gain, capital.loss, hours.per.week

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] "Training set row count: 26048"

[1] "Testing set row count: 6513"

In [4]:
head(df_train)

age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K
54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K
34,Private,216864,HS-grad,9,Divorced,Other-service,Unmarried,White,Female,0,3770,45,United-States,<=50K


# Tidying/Cleaning The Data & Selecting Columns
We can begin by tidying up the data. Something we noticed immediately that there are a lot of categorical columns. To tackle this, we plan to make buckets of categories and then turn them into dummy variables. For example, we can turn __sex__ into a dummy variable where 0 = male and 1 = female, make dummy variables by bucketting different things such as paid/unpaid in __workclass__, or making a dummy variable for every unique value. 

These are the columns we plan to use and how we plan to tackle them to make them usable in data: 
- **Age**: No changes (either than scaling/imputation)
- **Workclass**: Make a dummy variable for paid/unpaid
- **Education**: Simplified to a dummy variable of if College Graduate or Not
- **Occupation**: Make a dummy variable for all the occupations
- **Relationship**: Simplified a dummy variable of Married or Not
- **Sex**: Transformed into a Dummy Variable
- **Capital Gain**: No changes (either than scaling/imputation)
- **Capital Loss**: No changes (either than scaling/imputation)
- **Hours Per Week**: No changes (either than scaling/imputation)
- **Native Country**: Evaluate what country has the most people with income > 50k and then make a dummy variable for being in this country or not 

**Dropped Columns & Reasoning**: 
- fnlwgt: Unclear how this relates
- education.num: Already have education
- race: Don"t believe this is significant to the study so removing to avoid overfitting 
- marital.status: We think it"s similar to relationship so we remove to avoid overfitting 


This is also some tidying to do. We can initially see that there are some missing values in __workclass__ and __occupation__ represented as ?. Since we believe these are extremely important roles to guess income category, we remove any rows that don't have data for this. 

In [5]:
filtered_df <- df_train |>
  select(age, workclass, education, occupation, relationship, sex,
         capital.gain, capital.loss, hours.per.week, native.country, income) |>
  filter(workclass != "?" & occupation != "?")

Then, we can begin by making dummy variables in the data using the above choices. Alongside the code to mutate the dataframes, we will also show the unique data and verification of our dummy variables.

### Binary Preprocessing

Our initital thought for native country is that since the U.S probably has the most people with income >= 50k by a large margin. Therefore we wanted to simplify this variable to be a dummy variable of American vs not. This is how we verified our conclusion with code:

In [16]:
descending_df <- filtered_df %>%
  filter(income == "<=50K") %>% 
  group_by(native.country) %>%
  summarize(rich = n()) %>%
  ungroup() %>%
  mutate(total_count = sum(rich)) %>%
  mutate(rich_proportion = rich / total_count) %>%
  arrange(desc(rich_proportion))

head(descending_df)

native.country,rich,total_count,rich_proportion
<chr>,<int>,<int>,<dbl>
United-States,16431,18431,0.891487168
Mexico,447,18431,0.024252618
?,330,18431,0.017904617
Philippines,94,18431,0.005100103
Puerto-Rico,83,18431,0.004503283
El-Salvador,68,18431,0.003689436


To make the dummy variables:

In [ ]:
paste0("Relationship: ", list(unique(filtered_df$relationship)))
paste0("Sex: ", list(unique(filtered_df$sex)))
paste0("Workclass: ", list(unique(filtered_df$workclass)))
paste0("Country: ", list(unique(filtered_df$native.country)))

# Making a function to be used later on testing data 
preprocess_binary <- function(df){
    transformed_df <- df |>
        mutate(is_married = ifelse((relationship == "Husband" | relationship == "Wife" |relationship == "Wife"),1,0)) |>
        mutate(sex_dummy = ifelse(sex == "Female", 1, 0)) |>
        mutate(being_paid = ifelse(workclass != "Self-emp-not-inc" & workclass != "Without-pay" & workclass != "Never-worked", 1, 0)) |>
        mutate(is_american = ifelse(native.country == "United-States", 1, 0))
    return(transformed_df)
}

binary_df <- preprocess_binary(filtered_df)
binary_df |> 
    select(relationship, is_married, sex, sex_dummy, workclass, being_paid, native.country, is_american) |>
    slice(20:30)

[1] "Relationship: c(\"Not-in-family\", \"Own-child\", \"Unmarried\", \"Husband\", \"Wife\", \"Other-relative\")"

[1] "Sex: c(\"Female\", \"Male\")"

[1] "Workclass: c(\"Private\", \"Self-emp-not-inc\", \"Local-gov\", \"Self-emp-inc\", \"State-gov\", \"Federal-gov\", \"Without-pay\")"

[1] "Country: c(\"United-States\", \"Mexico\", \"Greece\", \"Vietnam\", \"?\", \"Trinadad&Tobago\", \"Canada\", \"South\", \"Holand-Netherlands\", \"Philippines\", \"China\", \"Poland\", \"Iran\", \"England\", \"Cambodia\", \"Dominican-Republic\", \"Hungary\", \"Puerto-Rico\", \"Japan\", \"El-Salvador\", \"Nicaragua\", \"India\", \"Haiti\", \"Cuba\", \"Germany\", \"Ireland\", \"Portugal\", \"Columbia\", \"Guatemala\", \"Italy\", \"Ecuador\", \"Laos\", \"France\", \"Taiwan\", \"Peru\", \"Jamaica\", \"Honduras\", \"Yugoslavia\", \"Thailand\", \"Outlying-US(Guam-USVI-etc)\", \"Hong\", \"Scotland\"\n)"

relationship,is_married,sex,sex_dummy,workclass,being_paid,native.country,is_american
<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>
Husband,1,Male,0,Private,1,United-States,1
Husband,1,Male,0,Self-emp-not-inc,0,United-States,1
Husband,1,Male,0,Self-emp-inc,1,United-States,1
Not-in-family,0,Female,1,Private,1,United-States,1
Not-in-family,0,Male,0,Local-gov,1,United-States,1
Unmarried,0,Male,0,Local-gov,1,United-States,1
Not-in-family,0,Male,0,Private,1,United-States,1
Not-in-family,0,Female,1,State-gov,1,United-States,1
Not-in-family,0,Male,0,Self-emp-inc,1,United-States,1


### Dummy Variables for Every Occupation
Since occupations are in buckets already, we can make a dummy variable for all of them

In [ ]:
# Making a function to use later
unique_occupations <- as.list(unique(binary_df$occupation))
paste0("Occupations: ", list(unique(binary_df$occupation)))

preprocess_occupation <- function(df, occupations){
    for (occ in occupations) {
        col_name <- gsub("[^a-zA-Z0-9 ]", ".", tolower(occ))
        df[[col_name]] <- as.integer(df$occupation == occ)
    }
    return(df)
}

occupations_df <- preprocess_occupation(binary_df, unique_occupations) 

occupations_df[, c(which(names(occupations_df) == 'occupation'), tail(seq_along(processed_df), length(unique_occupations)))] |>
head(5)

[1] "Occupations: c(\"Exec-managerial\", \"Prof-specialty\", \"Other-service\", \"Adm-clerical\", \"Sales\", \"Transport-moving\", \"Craft-repair\", \"Farming-fishing\", \"Handlers-cleaners\", \"Machine-op-inspct\", \"Protective-serv\", \"Tech-support\", \"Priv-house-serv\", \"Armed-Forces\")"

occupation,income,is_married,sex_dummy,being_paid,is_american,exec.managerial,prof.specialty,other.service,adm.clerical,sales,transport.moving,craft.repair,farming.fishing,handlers.cleaners
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
Exec-managerial,<=50K,0,1,1,1,1,0,0,0,0,0,0,0,0
Prof-specialty,<=50K,0,1,1,1,0,1,0,0,0,0,0,0,0
Other-service,<=50K,0,1,1,1,0,0,1,0,0,0,0,0,0
Adm-clerical,<=50K,0,0,1,1,0,0,0,1,0,0,0,0,0
Sales,<=50K,0,1,1,1,0,0,0,0,1,0,0,0,0


Keeping only dummy variables:

In [ ]:
tidy_df <- occupations_df |>
    select(-c(occupation, workclass, education, occupation, relationship, sex, native.country))

head(tidy_df, 3)

age,capital.gain,capital.loss,hours.per.week,native.country,income,is_married,sex_dummy,being_paid,is_american,...,sales,transport.moving,craft.repair,farming.fishing,handlers.cleaners,machine.op.inspct,protective.serv,tech.support,priv.house.serv,armed.forces
<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,...,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
82,0,4356,18,United-States,<=50K,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
41,0,3900,40,United-States,<=50K,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
34,0,3770,45,United-States,<=50K,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0


All in all, the processing can be explained by this one function:

In [ ]:
process_df <- function(df){
    filtered_df <- df |>
    select(age, workclass, education, occupation, relationship, sex,
            capital.gain, capital.loss, hours.per.week, native.country, income) |>
    filter(workclass != "?" & occupation != "?")

    binary_df <- preprocess_binary(filtered_df)

    unique_occupations <- as.list(unique(binary_df$occupation))
    occupations_df <- preprocess_occupation(binary_df, unique_occupations) 
    new_cols_df <- select(occupations_df, -c(occupation, workclass, education, occupation, relationship, sex, native.country))

    return(new_cols_df)
}
processed_df_train <- process_df(df_train)
print(all.equal(processed_df_train, tidy_df) & all.equal(colnames(processed_df_train), colnames(tidy_df)))

[1] TRUE


In [ ]:
scale_impute_df <- function(df){
  original_num_cols <- c("age", "capital.gain", "capital.loss", "hours.per.week")

  impute_scale_recipe <- recipe(income ~ ., data = df) |>
    step_scale(original_num_cols) |>
    step_center(original_num_cols) |>
    step_impute_mean(original_num_cols) |>
    prep()

  transformed_df <- bake(impute_scale_recipe, df)
  return(transformed_df)
}

processed_df <- scale_impute_df(new_cols_df)
head(processed_df, 5)

age,capital.gain,capital.loss,hours.per.week,native.country,is_married,sex_dummy,being_paid,is_american,exec.managerial,...,craft.repair,farming.fishing,adm.clerical,handlers.cleaners,protective.serv,tech.support,transport.moving,priv.house.serv,armed.forces,income
<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,...,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<fct>
3.3267000,-0.1485049,10.543798,-1.91502768,United-States,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,<=50K
1.1897266,-0.1485049,9.417188,-0.07788827,United-States,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,<=50K
0.1975604,-0.1485049,9.417188,-0.07788827,United-States,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,<=50K
-0.3366830,-0.1485049,9.096005,0.33964342,United-States,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,<=50K
2.2582133,-0.1485049,8.881060,-0.07788827,United-States,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,<=50K
